Goal: Examine correlations between global,socially significant events and Bitcoin blockchain metrics (block congestion, price movement, metadata notes) over time.
Utilizing publick bitcoin blockchain datasets stored in bigquery
to begin, I installed the following libraries:
pip install google-cloud-bigquery pandas pyarrow

Once I started trying to communicate with bigquery database, I had to also install:
pip install db-dtypes
to help pandas understand how to display data types used in SQL like "DATE, TIMESTAMP, STRUCT,etc.

Tables used:
 
A. bigquery-public-data.crypto_bitcoin.blocks table
B. bigquery-public-data.crypto_bitcoin.transactions table



In [3]:
%pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import bigquery
import os
#Set the environement variable for key file (telling python where my login key file is)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Credentials/capstone-project-461516-4b2a88c5f4cd.json"
#Start the BigQuery client using my key
client = bigquery.Client()



Due to data constraints, I decided to refine my dataset to focus on the 6 months period before/after covid lockdown took place in the US (Jan 1st 2020~June 30th 2020)
Due to issues to the original datasets I was working with (not having data for the date ranges I wanted), I need to change and update the list of tables I'm using



In [ ]:
# Checking if the new table "crypto_bitcoin.blocks" has information for the timeperiod I'm looking for:
query_sample = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.blocks`
LIMIT 5
"""

df_sample = client.query(query_sample).to_dataframe()
df_sample.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   hash               5 non-null      object             
 1   size               5 non-null      Int64              
 2   stripped_size      5 non-null      Int64              
 3   weight             5 non-null      Int64              
 4   number             5 non-null      Int64              
 5   version            5 non-null      Int64              
 6   merkle_root        5 non-null      object             
 7   timestamp          5 non-null      datetime64[us, UTC]
 8   timestamp_month    5 non-null      dbdate             
 9   nonce              5 non-null      object             
 10  bits               5 non-null      object             
 11  coinbase_param     5 non-null      object             
 12  transaction_count  5 non-null      Int64              

In [7]:
# Checking if the new table "crypto_bitcoin.blocks" has information for the timeperiod I'm looking for:
query_check_range = """
SELECT
  MIN(timestamp) AS start_date,
  MAX(timestamp) AS end_date
FROM `bigquery-public-data.crypto_bitcoin.blocks`
"""

df_range = client.query(query_check_range).to_dataframe()
df_range

,start_date,end_date
0,2009-01-03 18:15:05+00:00,2025-07-28 22:58:15+00:00


so we made sure the information in the crypto_bitcoin.blocks table has block information starting 2009~2025 July 28th.
now checking the type of information obtained on the crypto_bitcoin.transactions table:

In [ ]:
# checking the type of information obtained on the crypto_bitcoin.transactions table:
query_preview_tx = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.transactions`
LIMIT 5
"""
df_tx = client.query(query_preview_tx).to_dataframe()
df_tx.info()
df_tx.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   hash                   5 non-null      object             
 1   size                   5 non-null      Int64              
 2   virtual_size           5 non-null      Int64              
 3   version                5 non-null      Int64              
 4   lock_time              5 non-null      Int64              
 5   block_hash             5 non-null      object             
 6   block_number           5 non-null      Int64              
 7   block_timestamp        5 non-null      datetime64[us, UTC]
 8   block_timestamp_month  5 non-null      dbdate             
 9   input_count            5 non-null      Int64              
 10  output_count           5 non-null      Int64              
 11  input_value            5 non-null      object             
 12

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,efb7e4929cd066cd6d8ffa7f16d8efb1e1fc6fc04c2eee...,342,180,2,907621,00000000000000000001d5629d2923c7e89c1bcb621ea1...,907623,2025-07-28 22:58:15+00:00,2025-07-01,2,1,10104420.000000000,10101847.000000000,False,2573.000000000,"[{'index': 0, 'spent_transaction_hash': '6bc90...","[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
1,a89d90afff1f6e572505b95bc1d34d6f910d9c9b7cd655...,223,142,2,0,00000000000000000001d5629d2923c7e89c1bcb621ea1...,907623,2025-07-28 22:58:15+00:00,2025-07-01,1,2,12829.000000000,11125.000000000,False,1704.000000000,"[{'index': 0, 'spent_transaction_hash': '72f6a...","[{'index': 0, 'script_asm': 'OP_HASH160 331cb0..."
2,a6c7489e3c170a8f09d6e80c8b24adc75a820c483d1216...,240,155,1,0,00000000000000000001d5629d2923c7e89c1bcb621ea1...,907623,2025-07-28 22:58:15+00:00,2025-07-01,1,2,1047486.000000000,1041364.000000000,False,6122.000000000,"[{'index': 0, 'spent_transaction_hash': '8be9e...","[{'index': 0, 'script_asm': 'OP_HASH160 9e8ce6..."
3,59daef27c47858af00aaa78ea18caf784aab1bf4c85053...,223,142,2,0,00000000000000000001d5629d2923c7e89c1bcb621ea1...,907623,2025-07-28 22:58:15+00:00,2025-07-01,1,2,3767744.000000000,3765697.000000000,False,2047.000000000,"[{'index': 0, 'spent_transaction_hash': '957fa...","[{'index': 0, 'script_asm': '0 3f09c0894e5e006..."
4,4bd7506a444409e7024de279d3bb129d3398ac295c0344...,222,141,2,0,00000000000000000001d5629d2923c7e89c1bcb621ea1...,907623,2025-07-28 22:58:15+00:00,2025-07-01,1,2,109270302.000000000,109268163.000000000,False,2139.000000000,"[{'index': 0, 'spent_transaction_hash': '5f5a9...","[{'index': 0, 'script_asm': '0 705fde574d14136..."


In [9]:
# Now checking to make sure there's enough data during Jan-June 2020 in the transactions table:
query_row_count = """
SELECT COUNT(*) as transaction_count
FROM `bigquery-public-data.crypto_bitcoin.transactions`
WHERE block_timestamp BETWEEN '2020-01-01' AND '2020-06-30'
"""
df_row_count = client.query(query_row_count).to_dataframe()
print(df_row_count)

   transaction_count
0           54643498


In [10]:
# doing a small check to see if I can join things on the blocks and transactions table:
query_join_test = """
SELECT
  t.hash AS transaction_id,
  t.block_number,
  b.hash AS block_hash,
  b.timestamp AS block_time,
  t.fee,
  t.input_value,
  t.output_value
FROM
  `bigquery-public-data.crypto_bitcoin.transactions` t
JOIN
  `bigquery-public-data.crypto_bitcoin.blocks` b
ON
  t.block_number = b.number
WHERE
  t.block_timestamp BETWEEN '2020-01-01' AND '2020-06-30'
LIMIT 10
"""
df_join_test = client.query(query_join_test).to_dataframe()
df_join_test.head()

,transaction_id,block_number,block_hash,block_time,fee,input_value,output_value
0,cd0ba3f8235c7052f2f3d68a26236e4a682eef77240b0b...,610833,0000000000000000000c2b6bd76a87f1957d3aca4f8dee...,2020-01-01 19:41:08+00:00,25000.000000000,20034996.000000000,20009996.000000000
1,fd9eaa6ac1fbc02843ae29e88fb74e0135e8dd2a0c098e...,612791,00000000000000000000c51b41435410e0bc600c20f4a3...,2020-01-14 11:31:02+00:00,25000.000000000,30000000.000000000,29975000.000000000
2,9ad4f0443c4cef448a7fda3eb28417ad26b52cf7d12374...,612428,0000000000000000000c48efb43f8f400b4e3922622f33...,2020-01-12 00:09:41+00:00,15000.000000000,6092990.000000000,6077990.000000000
3,49b30138cc68f677be31a0a6d5ae49e5a734a9f5aca4a4...,612715,000000000000000000086ce680c9916f180539d402340e...,2020-01-13 22:33:52+00:00,31200.000000000,130000.000000000,98800.000000000
4,0a0993c49aec974fa0a563ef2042bfdbeb00f7f41924a8...,613640,0000000000000000000c9f22305fd14cbe72b23b2d9c24...,2020-01-20 03:57:36+00:00,31200.000000000,130000.000000000,98800.000000000


In [11]:
query_blocks_covid = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.blocks`
WHERE timestamp BETWEEN '2020-01-01' AND '2020-06-30'
"""
df_blocks = client.query(query_blocks_covid).to_dataframe()
df_blocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26253 entries, 0 to 26252
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   hash               26253 non-null  object             
 1   size               26253 non-null  Int64              
 2   stripped_size      26253 non-null  Int64              
 3   weight             26253 non-null  Int64              
 4   number             26253 non-null  Int64              
 5   version            26253 non-null  Int64              
 6   merkle_root        26253 non-null  object             
 7   timestamp          26253 non-null  datetime64[us, UTC]
 8   timestamp_month    26253 non-null  dbdate             
 9   nonce              26253 non-null  object             
 10  bits               26253 non-null  object             
 11  coinbase_param     26253 non-null  object             
 12  transaction_count  26253 non-null  Int64      

Since I found out the bitcoin transactions table for March of 2020 adds up to 10 Million rows, I decided to extract a sample of 100K rows of data

In [ ]:
# Getting a sample (100k rows) of data from the bitcoin transactions table for March 2020:
query_transactions_march_2020 = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.transactions`
WHERE block_timestamp BETWEEN '2020-03-01' AND '2020-03-31 23:59:59'
LIMIT 100000
"""
df_transactions = client.query(query_transactions_march_2020).to_dataframe()
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   hash                   100000 non-null  object             
 1   size                   100000 non-null  Int64              
 2   virtual_size           100000 non-null  Int64              
 3   version                100000 non-null  Int64              
 4   lock_time              100000 non-null  Int64              
 5   block_hash             100000 non-null  object             
 6   block_number           100000 non-null  Int64              
 7   block_timestamp        100000 non-null  datetime64[us, UTC]
 8   block_timestamp_month  100000 non-null  dbdate             
 9   input_count            100000 non-null  Int64              
 10  output_count           100000 non-null  Int64              
 11  input_value            99962 non-null   

In [31]:
(df_transactions['block_timestamp'].min(), df_transactions['block_timestamp'].max())

(Timestamp('2020-03-01 00:04:03+0000', tz='UTC'),
 Timestamp('2020-03-31 23:51:06+0000', tz='UTC'))

In [8]:
# SQLite start
import sqlite3


In [10]:
# Connecting to a new SQLite database to create a file for my dataset
connection = sqlite3.connect("bitcoin_data.db")

In [12]:
# Saving the blocks DataFrame into the database as a new table called "blocks"
table_name = "blocks"
data = df_blocks
# Writing the data in df_blocks to the database
data.to_sql(
        name = table_name,
        con = connection,
        if_exists = "replace",
        index = False
)


NameError: name 'df_blocks' is not defined

In [3]:
#Testing my code to see if it is working:
query1 = """
    SELECT block_id
    FROM `bigquery-public-data.bitcoin_blockchain.blocks`
    LIMIT 5
"""


df = client.query(query1).to_dataframe()
print(df)

                                            block_id
0  00000000000000000069564d0540f7d160fc7400e03c49...
1  000000000000000000022ab0c5740584d31006ddcd1e20...
2  000000000000000001628d23a7ba194fe9498676960947...
3  00000000000000000025a758640f78eff1fe5bc472ff03...
4  0000000000000000004769939c52cfa8e8f1585b580080...


C:\Users\smby2\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
import sys
print(sys.executable)


c:\Python313\python.exe


In [5]:
query2 = """
    SELECT *
    FROM `bigquery-public-data.bitcoin_blockchain.blocks`
    LIMIT 5
"""
print(df.columns)

Index(['block_id'], dtype='object')


In [6]:
query3 = """
    SELECT *
    FROM `bigquery-public-data.bitcoin_blockchain.transactions`
    LIMIT 5
"""
print(df.columns)

Index(['block_id'], dtype='object')


In [7]:
query4 = """
    SELECT 
      o.script_asm,
      o.block_number,
      o.value,
      o.transaction_hash
    FROM `bigquery-public-data.crypto_bitcoin.outputs` AS o
    WHERE o.script_asm LIKE '%OP_RETURN%'
    LIMIT 10
"""

df = client.query(query4).to_dataframe()
print(df.head())


                                          script_asm  block_number value  \
0  OP_RETURN 58325bbc0b0f83bf38d794ddddcd41ac0944...        906690  0E-9   
1  OP_RETURN 58325bbc0b0f83bf38d794ddddcd41ac0944...        906690  0E-9   
2  OP_RETURN 00034db3c093ea8ba03987aed40813c04acb...        906690  0E-9   
3  OP_RETURN 58325bbc0b0f83bf38d794ddddcd41ac0944...        906690  0E-9   
4  OP_RETURN 58325bbc0b0f83bf38d794ddddcd41ac0944...        906690  0E-9   

                                    transaction_hash  
0  eb62fd85aedcd9fec45bad73d048a1e8d0f48ae17ababd...  
1  124a375deda998fac0b70c561b83cc629117b96ff46b3f...  
2  5161061aa560fb6b9541d97b34c05bf61bb0ff60ab5abb...  
3  71587df89da762662f7e44011dbf2c362f600a7c0c896f...  
4  9abf76d1f14e37a54a2a462fa332c46f3167ff9aa7c39c...  


C:\Users\smby2\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [8]:
query5 = """
    SELECT *
    FROM `bigquery-public-data.crypto_bitcoin.outputs` AS o
    WHERE o.script_asm LIKE '%OP_RETURN%'
    LIMIT 10
"""

df = client.query(query5).to_dataframe()
print(df.columns)

Index(['transaction_hash', 'block_hash', 'block_number', 'block_timestamp',
       'index', 'script_asm', 'script_hex', 'required_signatures', 'type',
       'addresses', 'value'],
      dtype='object')


C:\Users\smby2\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Stopped here as of 5/31. when I left off, I was able to see that tables:
A. bigquery-public-data.bitcoin_blockchain.blocks table
B. bigquery-public-data.bitcoin_blockchain.transactions table
C. bigquery-public-data.crypto_bitcoin.outputs table
cointained the data I needed to connect block timestamp to messages hidden in OP_RETURN column (in the outputs table) 
1. blocks → has block_hash and block_timestamp
2. transactions → has both transaction_hash and block_hash
3. outputs → has transaction_hash



